# PyTorch 基礎實戰

## 學習目標
- 掌握 PyTorch Tensor 操作
- 理解自動微分 (Autograd) 機制
- 學會建立自定義 Dataset 和 DataLoader
- 熟悉 nn.Module 模型建構

## 參考資源
- [李宏毅 ML 2021](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [PyTorch 官方教程](https://pytorch.org/tutorials/)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# 檢查 PyTorch 版本和 CUDA
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

---
## Part 1: Tensor 基礎操作

Tensor 是 PyTorch 的核心資料結構，類似 NumPy 的 ndarray，但可以在 GPU 上運算。

In [ ]:
# ========== Tensor 建立 ==========

# 從 Python list 建立
x = torch.tensor([1, 2, 3, 4])
print(f"從 list 建立: {x}")

# 從 NumPy 建立
np_array = np.array([[1, 2], [3, 4]])
x_np = torch.from_numpy(np_array)
print(f"從 NumPy 建立:\n{x_np}")

# 特殊 Tensor
zeros = torch.zeros(2, 3)
ones = torch.ones(2, 3)
rand = torch.rand(2, 3)  # [0, 1) 均勻分布
randn = torch.randn(2, 3)  # 標準正態分布

print(f"\n隨機 Tensor:\n{randn}")

# 指定資料類型和設備
x_float = torch.tensor([1, 2, 3], dtype=torch.float32)
x_gpu = torch.rand(2, 3).to(device)
print(f"\nGPU Tensor 設備: {x_gpu.device}")

In [ ]:
# ========== Tensor 屬性 ==========

x = torch.randn(3, 4, 5)

print(f"形狀 (shape): {x.shape}")
print(f"維度數 (dim): {x.dim()}")
print(f"資料類型 (dtype): {x.dtype}")
print(f"設備 (device): {x.device}")
print(f"元素數量: {x.numel()}")
print(f"是否需要梯度: {x.requires_grad}")

In [ ]:
# ========== Tensor 運算 ==========

a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

# 元素運算
print(f"加法: \n{a + b}")
print(f"乘法 (元素): \n{a * b}")
print(f"矩陣乘法: \n{a @ b}")
print(f"矩陣乘法 (等價): \n{torch.matmul(a, b)}")

# 聚合運算
print(f"\n總和: {a.sum()}")
print(f"平均: {a.mean()}")
print(f"最大值: {a.max()}")
print(f"沿軸求和: {a.sum(dim=0)}")
print(f"沿軸最大: {a.max(dim=1)}")

In [ ]:
# ========== Tensor 形狀操作 ==========

x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"原始: {x.shape}")

# reshape / view
x_reshaped = x.view(3, 4)
print(f"reshape 為 (3,4):\n{x_reshaped}")

# 自動推斷維度
x_auto = x.view(2, -1)  # -1 自動計算
print(f"reshape 為 (2,-1): {x_auto.shape}")

# 轉置
x_t = x_reshaped.T
print(f"轉置: {x_t.shape}")

# 增加/移除維度
x_unsq = x.unsqueeze(0)  # [12] -> [1, 12]
x_sq = x_unsq.squeeze(0)  # [1, 12] -> [12]
print(f"unsqueeze: {x_unsq.shape}")
print(f"squeeze: {x_sq.shape}")

# 展平
x_flat = x_reshaped.flatten()
print(f"flatten: {x_flat.shape}")

In [ ]:
# ========== 索引與切片 ==========

x = torch.arange(20).view(4, 5)
print(f"原始:\n{x}")

# 基本索引
print(f"x[0]: {x[0]}")
print(f"x[0, 0]: {x[0, 0]}")
print(f"x[:, 0]: {x[:, 0]}")
print(f"x[1:3, 2:4]:\n{x[1:3, 2:4]}")

# 布林索引
mask = x > 10
print(f"x > 10:\n{mask}")
print(f"x[x > 10]: {x[mask]}")

# 進階索引
indices = torch.tensor([0, 2, 3])
print(f"x[indices]:\n{x[indices]}")

---
## Part 2: 自動微分 (Autograd)

PyTorch 的自動微分系統讓我們可以自動計算梯度，這是訓練神經網路的核心。

In [ ]:
# ========== 基本 Autograd ==========

# 建立需要梯度的 tensor
x = torch.tensor([2.0, 3.0], requires_grad=True)
print(f"x: {x}")
print(f"x.requires_grad: {x.requires_grad}")

# 前向計算
y = x ** 2 + 3 * x + 1
print(f"y = x² + 3x + 1: {y}")

# 計算總和以得到純量
loss = y.sum()
print(f"loss = sum(y): {loss}")

# 反向傳播
loss.backward()

# dy/dx = 2x + 3
print(f"x.grad (∂loss/∂x): {x.grad}")
print(f"驗證: 2*x + 3 = {2*x.detach() + 3}")

In [ ]:
# ========== 計算圖與梯度累積 ==========

x = torch.tensor([1.0], requires_grad=True)

# 多次反向傳播會累積梯度
for i in range(3):
    y = x * 2
    y.backward()
    print(f"第 {i+1} 次: x.grad = {x.grad}")

print("\n梯度會累積！訓練時記得清零。")

# 正確做法：清零梯度
x = torch.tensor([1.0], requires_grad=True)
for i in range(3):
    if x.grad is not None:
        x.grad.zero_()  # 清零梯度
    y = x * 2
    y.backward()
    print(f"清零後第 {i+1} 次: x.grad = {x.grad}")

In [ ]:
# ========== 停止梯度追蹤 ==========

x = torch.randn(3, requires_grad=True)

# 方法 1: with torch.no_grad()
with torch.no_grad():
    y = x * 2
    print(f"no_grad 內: y.requires_grad = {y.requires_grad}")

# 方法 2: .detach()
y = x.detach() * 2
print(f"detach: y.requires_grad = {y.requires_grad}")

# 方法 3: @torch.no_grad() 裝飾器
@torch.no_grad()
def inference(x):
    return x * 2

y = inference(x)
print(f"裝飾器: y.requires_grad = {y.requires_grad}")

---
## Part 3: Dataset 與 DataLoader

PyTorch 提供了 Dataset 和 DataLoader 來處理資料載入和批次處理。

In [ ]:
# ========== 自定義 Dataset ==========

class CustomDataset(Dataset):
    """
    自定義 Dataset 需要實現三個方法：
    - __init__: 初始化
    - __len__: 返回資料集大小
    - __getitem__: 根據索引返回樣本
    """
    
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        
        if self.transform:
            x = self.transform(x)
        
        return x, y

# 建立範例資料
X = torch.randn(100, 10)  # 100 個樣本，每個 10 維
y = torch.randint(0, 2, (100,))  # 二分類標籤

dataset = CustomDataset(X, y)
print(f"資料集大小: {len(dataset)}")
print(f"第一個樣本: {dataset[0][0].shape}, 標籤: {dataset[0][1]}")

In [ ]:
# ========== DataLoader ==========

# 建立 DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=16,       # 批次大小
    shuffle=True,        # 是否打亂
    num_workers=0,       # 並行載入的進程數
    drop_last=True,      # 丟棄最後不完整的批次
)

print(f"批次數: {len(dataloader)}")

# 迭代 DataLoader
for batch_idx, (batch_x, batch_y) in enumerate(dataloader):
    print(f"Batch {batch_idx}: X shape = {batch_x.shape}, y shape = {batch_y.shape}")
    if batch_idx >= 2:
        break

In [ ]:
# ========== 實用技巧：訓練/驗證分割 ==========

from torch.utils.data import random_split

# 分割資料集
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"訓練集: {len(train_dataset)}")
print(f"驗證集: {len(val_dataset)}")

# 分別建立 DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

---
## Part 4: nn.Module 模型建構

所有 PyTorch 模型都繼承自 nn.Module。

In [ ]:
# ========== 基本模型定義 ==========

class SimpleNN(nn.Module):
    """
    簡單的前饋神經網路
    
    必須實現:
    - __init__: 定義層
    - forward: 定義前向傳播
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()  # 必須調用父類初始化
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# 建立模型
model = SimpleNN(input_dim=10, hidden_dim=64, output_dim=2)
print(model)

# 測試前向傳播
x = torch.randn(4, 10)
output = model(x)
print(f"\n輸入: {x.shape}")
print(f"輸出: {output.shape}")

In [ ]:
# ========== 使用 nn.Sequential ==========

# 快速建立簡單模型
model_seq = nn.Sequential(
    nn.Linear(10, 64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 2),
)

print("Sequential 模型:")
print(model_seq)

# 測試
output = model_seq(x)
print(f"\n輸出: {output.shape}")

In [ ]:
# ========== 模型參數 ==========

# 查看所有參數
print("模型參數:")
for name, param in model.named_parameters():
    print(f"  {name}: {param.shape}")

# 計算總參數量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n總參數: {total_params:,}")
print(f"可訓練參數: {trainable_params:,}")

In [ ]:
# ========== 模型狀態 ==========

# 訓練模式 vs 評估模式
model.train()  # 啟用 Dropout, BatchNorm 使用批次統計
print(f"訓練模式: model.training = {model.training}")

model.eval()   # 停用 Dropout, BatchNorm 使用累積統計
print(f"評估模式: model.training = {model.training}")

# 移動模型到 GPU
model = model.to(device)
print(f"\n模型設備: {next(model.parameters()).device}")

---
## Part 5: 完整訓練流程

把所有元素組合成完整的訓練流程。

In [ ]:
# ========== 完整訓練範例 ==========

def train_model(model, train_loader, val_loader, epochs=10, lr=1e-3):
    """
    標準 PyTorch 訓練流程
    """
    model = model.to(device)
    
    # 優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    # 損失函數
    criterion = nn.CrossEntropyLoss()
    
    # 學習率調度器
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    
    history = {'train_loss': [], 'val_loss': [], 'val_acc': []}
    
    for epoch in range(epochs):
        # ========== 訓練階段 ==========
        model.train()
        train_loss = 0
        
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # 1. 清零梯度
            optimizer.zero_grad()
            
            # 2. 前向傳播
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            
            # 3. 反向傳播
            loss.backward()
            
            # 4. 更新參數
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        
        # ========== 驗證階段 ==========
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():  # 關閉梯度計算
            for batch_x, batch_y in val_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += batch_y.size(0)
                correct += predicted.eq(batch_y).sum().item()
        
        val_loss /= len(val_loader)
        val_acc = correct / total
        
        # 更新學習率
        scheduler.step()
        
        # 記錄歷史
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print(f"  LR: {scheduler.get_last_lr()[0]:.6f}")
    
    return history

# 訓練
model = SimpleNN(input_dim=10, hidden_dim=64, output_dim=2)
history = train_model(model, train_loader, val_loader, epochs=10)

In [ ]:
# ========== 繪製訓練曲線 ==========

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history['val_acc'])
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## Part 6: 模型保存與載入

In [ ]:
# ========== 保存與載入 ==========

# 方法 1: 只保存權重（推薦）
torch.save(model.state_dict(), 'model_weights.pt')

# 載入權重
new_model = SimpleNN(input_dim=10, hidden_dim=64, output_dim=2)
new_model.load_state_dict(torch.load('model_weights.pt'))
print("權重載入成功！")

# 方法 2: 保存完整檢查點（包含優化器狀態）
checkpoint = {
    'epoch': 10,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict() if 'optimizer' in dir() else None,
    'loss': history['train_loss'][-1] if history['train_loss'] else None,
}
torch.save(checkpoint, 'checkpoint.pt')
print("檢查點保存成功！")

# 清理
import os
os.remove('model_weights.pt')
os.remove('checkpoint.pt')

---
## 🎯 總結

### PyTorch 核心概念

| 概念 | 說明 |
|------|------|
| **Tensor** | 多維數組，支持 GPU 加速 |
| **Autograd** | 自動微分，追蹤計算圖 |
| **Dataset** | 封裝資料，提供索引訪問 |
| **DataLoader** | 批次載入，支持多進程 |
| **nn.Module** | 模型基類，管理參數和前向傳播 |

### 訓練流程 Checklist

```
1. 準備資料
   □ 建立 Dataset
   □ 建立 DataLoader
   □ 分割訓練/驗證集

2. 建立模型
   □ 定義網路架構
   □ 移動到 GPU

3. 設定訓練
   □ 選擇優化器
   □ 選擇損失函數
   □ 設定學習率調度

4. 訓練循環
   □ 清零梯度 optimizer.zero_grad()
   □ 前向傳播 outputs = model(inputs)
   □ 計算損失 loss = criterion(outputs, labels)
   □ 反向傳播 loss.backward()
   □ 更新參數 optimizer.step()

5. 驗證與保存
   □ model.eval() + torch.no_grad()
   □ 保存最佳模型
```

### 下一步

前往 `supervised_learning/regression.ipynb` 學習如何用 PyTorch 實作迴歸任務！

---
## 練習題

### 練習 1: 實作一個簡單的線性回歸

**目標**: 使用 PyTorch 從零實作線性回歸 y = 2x + 3

In [ ]:
# 練習 1: 線性回歸

# 生成資料: y = 2x + 3 + noise
torch.manual_seed(42)
X = torch.linspace(-5, 5, 100).unsqueeze(1)  # [100, 1]
y_true = 2 * X + 3 + torch.randn_like(X) * 0.5

# 定義模型參數
w = torch.randn(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 訓練
lr = 0.01
epochs = 100
losses = []

for epoch in range(epochs):
    # 前向傳播
    y_pred = X * w + b
    loss = ((y_pred - y_true) ** 2).mean()  # MSE
    
    # 反向傳播
    loss.backward()
    
    # 更新參數（手動梯度下降）
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad
        
        # 清零梯度
        w.grad.zero_()
        b.grad.zero_()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, w = {w.item():.4f}, b = {b.item():.4f}")

print(f"\n學到的參數: y = {w.item():.2f}x + {b.item():.2f}")
print(f"真實參數: y = 2x + 3")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].scatter(X.numpy(), y_true.numpy(), alpha=0.5, label='Data')
axes[0].plot(X.numpy(), (X * w + b).detach().numpy(), 'r-', linewidth=2, label='Fitted')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Linear Regression')
axes[0].legend()

axes[1].plot(losses)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Training Loss')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 練習 2: 實作自定義 Dataset 處理圖像資料

**目標**: 建立一個處理合成圖像的 Dataset

In [ ]:
# 練習 2: 自定義圖像 Dataset

class SyntheticImageDataset(Dataset):
    """
    合成圖像資料集
    - 類別 0: 帶有水平條紋的圖像
    - 類別 1: 帶有垂直條紋的圖像
    """
    
    def __init__(self, num_samples=1000, img_size=28, transform=None):
        self.num_samples = num_samples
        self.img_size = img_size
        self.transform = transform
        
        # 預生成資料
        self.images = []
        self.labels = []
        
        for i in range(num_samples):
            if i % 2 == 0:
                # 水平條紋
                img = self._create_horizontal_stripes()
                label = 0
            else:
                # 垂直條紋
                img = self._create_vertical_stripes()
                label = 1
            
            self.images.append(img)
            self.labels.append(label)
    
    def _create_horizontal_stripes(self):
        img = torch.zeros(1, self.img_size, self.img_size)
        stripe_width = np.random.randint(2, 5)
        for i in range(0, self.img_size, stripe_width * 2):
            img[:, i:i+stripe_width, :] = 1.0
        img += torch.randn_like(img) * 0.1
        return img.clamp(0, 1)
    
    def _create_vertical_stripes(self):
        img = torch.zeros(1, self.img_size, self.img_size)
        stripe_width = np.random.randint(2, 5)
        for i in range(0, self.img_size, stripe_width * 2):
            img[:, :, i:i+stripe_width] = 1.0
        img += torch.randn_like(img) * 0.1
        return img.clamp(0, 1)
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

# 建立資料集
dataset = SyntheticImageDataset(num_samples=500)
print(f"資料集大小: {len(dataset)}")

# 視覺化樣本
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i in range(5):
    img, label = dataset[i * 2]
    axes[0, i].imshow(img.squeeze(), cmap='gray')
    axes[0, i].set_title(f'Class {label} (Horizontal)')
    axes[0, i].axis('off')
    
    img, label = dataset[i * 2 + 1]
    axes[1, i].imshow(img.squeeze(), cmap='gray')
    axes[1, i].set_title(f'Class {label} (Vertical)')
    axes[1, i].axis('off')

plt.suptitle('Synthetic Image Dataset')
plt.tight_layout()
plt.show()

### 練習 3: 使用自定義 Dataset 訓練 CNN

**目標**: 用簡單的 CNN 分類上面的條紋圖像

In [ ]:
# 練習 3: 訓練 CNN 分類條紋

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(32 * 7 * 7, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 28 -> 14
        x = self.pool(F.relu(self.conv2(x)))  # 14 -> 7
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 準備資料
train_dataset = SyntheticImageDataset(num_samples=800)
test_dataset = SyntheticImageDataset(num_samples=200)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# 訓練
model = SimpleCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # 測試
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{epochs}: Loss = {total_loss/len(train_loader):.4f}, Test Acc = {acc:.1f}%")

print(f"\n最終測試準確率: {acc:.1f}%")